# 546 (Optimization for Learning and Control) hw2

You are welcome (and encouraged) to work with others, but each individual must submit their own writeup.

You are welcome to use analytical and numerical computational tools; if you do, include the **commented** sourcecode in your writeup (e.g. the .ipynb file).

You are welcome to consult research articles and other materials; if you do, include a full citation in your writeup (e.g. the .ipynb file) and upload a .pdf of the article to Canvas alongside your homework submission.  **Your submission must be a product of your own work.**

In [9]:
import numpy as np
import math
import numpy.linalg as la
import scipy.linalg as sla
import matplotlib.pyplot as plt
import random
%matplotlib inline

## Markov decision process (MDP)

Let $(X,U,P,c)$ be an MDP, i.e.

* $X$ is a finite set of $n$ *states*
* $U$ is a finite set of $m$ *actions*
* $P:X\times U\rightarrow \Delta(X)$ is transition probability
* $c:X\times U\rightarrow \mathbb{R}$ is cost

where $\Delta(S) = \{p\in[0,1]^S : \sum_{s\in S} p(s) = 1\}$ is the set of probability distributions over the finite set $S$.

a. *Implement an algorithm that generates a random MDP given finite sets $X$ and $U$ (i.e. let $N = |X|$, $M = |U|$ be parameters that are easy to vary, and generate $P$ and $c$ randomly).*

In [42]:
def draw_from_dis(dis_list):
    s = np.random.choice(dis_list,p=dis_list)
    s = np.argmax(dis_list == s)
    return s
    
class MDP_generator:
    def __init__(self, X, U, get_P, get_c, p_0):
        # state set
        self.X = X
        # action set
        self.U = U
        # function takes current state and action return a distribution of next state (list of probabilities)
        self.P = get_P
        # cost function takes in current state and action
        self.c = get_c
        # current state (initialized randomly)
        s = draw_from_dis(p_0)
        self.x = s
        print('initial state: ' + str(self.x))
        
    def step(self, action):
        # action -- index over action list
        dis_list = self.P(self.x, action)
        #print dis_list
        s = draw_from_dis(dis_list)
        old_x = self.x
        self.x = s
        return self.x, self.c(old_x, action)

b. *Implement an algorithm that simulates an MDP (i.e. write a function that inputs transition probabilities $P$, cost $c$, initial state distribution $p_0$, policy $\pi:X\rightarrow\Delta(U)$, and time horizon $t$ and returns the state distribution $p_t$).*

In [43]:
def MDP_sim(X, U, get_P, get_c, p_0, pi, total_steps):
    env = MDP_generator(X, U, get_P, get_c, p_0)
    cost_history = []
    total_cost = 0
    #print 'init system state: ' + str(env.x)
    for i in range(total_steps):
        action_dis = pi(env.x)
        action = draw_from_dis(action_dis)
        next_x, cost = env.step(action)
        cost_history.append(cost)
        total_cost = total_cost + cost
        #print 'state: ' + str(next_x) + ' cost: ' + str(cost)
    #print 'total cost: ' + str(total_cost)
    return env.x, total_cost, cost_history

c. *Test your algorithm from (b.) (i.e. design one or more tests, explain why these tests are exhaustive, and provide summary statistics and/or visualizations that convince a skeptical reader that your algorithm is correct).*

Let us now consider the problem of minimizing the infinite-horizon discounted cost
$$J = \sum_{t=0}^\infty \gamma^t c_t,$$
where $\gamma\in(0,1)$ is a *discount factor* and $c_t = c(x_t,u_t)$ is the cost at time $t$.

Any policy $\pi : X\rightarrow\Delta(U)$ has an associated *value* $v^\pi : X\rightarrow\mathbb{R}$ defined by
$$\forall x\in X : v^\pi(x) = E[J \mid x = x_0]$$
that satisfies the *Bellman equation*
$$\forall x\in X : v^\pi(x) = \sum_{u\in U}\pi(u|x)\sum_{x^+\in X} P(x^+|x,u)\left[c(x,u) + \gamma \cdot v^\pi(x^+)\right].$$

In [81]:
gamma = 0.99

def discount_costs(c_history):
    # discounted costs
    discounted_c = np.zeros_like(c_history)
    running_add = 0
    for t in reversed(range(0, len(c_history))):
        running_add = running_add * gamma + c_history[t]
        discounted_c[t] = running_add
    return discounted_c

In [80]:
# X, U, P, c, p_0
X = 3 # 3 states
U = 3 # action for every possible system state

# X x U
P = np.array([[[0.1, 0.3, 0.6],[0.3, 0.4, 0.3],[0.0, 0.8, 0.2]],
              [[0.5, 0.0, 0.5],[0.0, 0.0, 1.0],[0.6, 0.1, 0.3]],
              [[0.2, 0.2, 0.6],[0.7, 0.1, 0.2],[0.4, 0.6, 0.0]]])

C = np.array([[4,3,5],[5,6,2],[7,2,1]])

def get_P(s, a):
    return P[s,a]
        
def get_c(s, a):
    return C[s,a]
        
def pi(s):
    if s == 0:
        return [0.2, 0.3, 0.5]
    elif s == 1:
        return [0.4, 0.5, 0.1]
    else:
        return [0.3, 0.1, 0.6]
        
p_0 = [0.3, 0.2, 0.5]

total_steps = 1000
fx,fc,his_ = MDP_sim(X, U, get_P, get_c, p_0, pi, total_steps)
np.sum(discount_costs(his_))

initial state: 2


19830

d. *Noting that $v^\pi$ appears linearly in this Bellman equation, implement a *policy evaluation* algorithm that computes $v^\pi$ using linear algebra (i.e. determing $L$ and $b$ such that $L v^\pi = b$ and use this equation to solve for $v^\pi$).*

For value $v^{\pi} = (I- \gamma P^{\pi})^{-1} C^{\pi}$

In [51]:
def get_v(x, P, C, gamma):
    v_list = np.dot(np.linalg.inv(np.identity(3) - np.dot(gamma, P[x,:])), C[x])
    return np.sum(v_list)

e. *Test your policy evaluation algorithm from (d.) (i.e. design one or more tests, explain why these tests are exhaustive, and provide summary statistics and/or visualizations that convince a skeptical reader that your algorithm is correct).*

In [71]:
get_v(2, P, C, gamma)

1117.9624881196899

f. *Using your policy evaluation algorithm from (d.), implement *value iteration* and *policy iteration* algorithms.*

g. *Test your value iteration and policy iteration algorithms from (f.) (i.e. design one or more tests, explain why these tests are exhaustive, and provide summary statistics and/or visualizations that convince a skeptical reader that your algorithm is correct).*

## linear quadratic regulation

Consider the linear-quadratic regulation (LQR) problem for the discrete-time linear time-invariant system
$$x^+ = A x + B u$$
with infinite-horizon cost
$$c(x,u) = \frac{1}{2}\sum_{t=0}^\infty x^T Q x + u^T R u \ dt$$
where  $Q = Q^T > 0$ and $R = R^T > 0$.  We know that the optimal policy is linear in state, 
$$u = - K x,$$
where $K = (R + B^T Y B)^{-1} (B^T Y A)$ and $Y$ satisfies the discrete algebraic Riccati equation
$$Y = A^T Y A - (A^T Y B)(R + B^T Y B)^{-1}(B^T Y A) + Q.$$

For this problem, we'll consider a one-dimensional system, so $x,u\in\mathbb{R}$, with the values $A, B, Q, R = 1$.

If the state distribution starts out Gaussian, 
$$x \sim \mathcal{N}(\xi, \Sigma),$$
and we apply zero input but add a state disturbance with zero mean and covariance $W$, then (so long as the disturbance is uncorrelated with the state) the state distribution after one step is Gaussian:
$$x^+ \sim \mathcal{N}(A\xi, A\Sigma A^T + W),$$
i.e. the covariance gets updated via
$$\Sigma^+ = A\Sigma A^T + W.$$
Since $A$ is stable, so long as $W$ is constant, this iteration will converge to a solution of the Lyapunov equation
$$S = A S A^T + W.$$
Also, since $A$ is stable, the steady-state mean is zero, thus we can determine the steady-state distribution in closed-form:
$$\lim_{t\rightarrow\infty} x(t) \sim \mathcal{N}(0, S).$$

In this problem, you'll make use of the facts above to build a finite MDP that approximates the LQR, solve the MDP using your algorithms from the previous problem, then make use of the facts above again to validate the solution you've obtained.

a. *Construct a finite MDP by discretizing the state and action spaces of the linear system and computing transition probabilities and single-stage costs to obtain  $(X,U,P,C)$ where:*

* $X$ is a finite set of $N$ *states*;
* $U$ is a finite set of $M$ *actions*;
* $P:X\times U\rightarrow \Delta(X)$ is transition probability;
* $C:X\times U\times X\rightarrow \mathbb{R}$ is single-stage cost.

b. *Compare the steady-state distribution of the MDP with the steady-state distribution of the linear system subject to zero-mean disturbance with covariance $W$.  Discuss how the comparison varies with respect to $|X|$.*

Now consider the problem of minimizing the infinite-horizon discounted cost
$$c = \sum_{t=0}^\infty \gamma^t C_t,$$
where $\gamma\in(0,1]$ is a *discount factor* and $C_t = C(x_t,u_t)$ is the cost at time $t$.

The un-discounted case $\gamma = 1$ corresponds exactly to LQR; unfortunately, policy iteration won't converge when $\gamma = 1$!

c. *Compare the optimal value and control for the MDP obtained using value or policy iteration with the optimal value and control for the original LQR problem.  Discuss how the comparison varies with respect to $|X|$, $|U|$, and $\gamma$.*